Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

## Описание данных 

- **date** – дата совершения события

- **event** - событие:

    * app_install – установка приложения
    * app_start – открыть приложения
    * registration – зарегистрироваться 
    * search – перейти на страницу поиска товаров (каталог)
    * open_item – открыть товар
    * choose_item – отправить товар в корзину
    * tap_basket – перейти в корзину
    * purchase – подтверждение покупки
- **gender** – пол пользователя

- **os_name** – платформа пользователя

- **city** – город пользователя

- **device_id** – идентификатор устройства пользователя

- **urm_source** – канал, с которого пришел пользователь

    * yandex-direct – Яндекс директ
    * google_ads – реклама в Google
    * vk_ads – реклама в ВК
    * instagram_ads – реклама в instagram
    * facebook_ads – реклама в facebook
    * referral – акция «приведи друга»
    * Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
- **purchase_sum** – стоимость покупки (при совершении события ‘purchase’)

### *Обратите внимание на следующее:*

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

In [21]:
import pandas as pd

In [133]:
df = pd.read_csv('KC_case_data.csv')
df

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads
...,...,...,...,...,...,...,...,...
2747963,2020-03-31,register,NaN,android,2984778,male,Saint-Petersburg,facebook_ads
2747964,2020-03-31,register,NaN,ios,27301864,male,Moscow,-
2747965,2020-03-31,register,NaN,ios,1294285,female,Saint-Petersburg,google_ads
2747966,2020-03-31,register,NaN,android,3010574,female,Saint-Petersburg,google_ads


In [4]:
df.date = pd.to_datetime(df.date, 
                         errors = 'ignore')
df.date

0         2020-01-01
1         2020-01-01
2         2020-01-01
3         2020-01-01
4         2020-01-01
             ...    
2747963   2020-03-31
2747964   2020-03-31
2747965   2020-03-31
2747966   2020-03-31
2747967   2020-03-31
Name: date, Length: 2747968, dtype: datetime64[ns]

## 1.MAU

In [5]:
df.groupby(pd.Grouper(key  = 'date', 
                      freq = '1M')) \
    .agg({'device_id' : 'nunique'})

,device_id
date,
2020-01-31,99161
2020-02-29,75032
2020-03-31,74623


## 2. Количество установок

In [6]:
df.query('event == "app_install"') \
    .groupby(pd.Grouper(key  = 'date', 
                        freq = '1M')) \
    .agg({'device_id' : 'count'})

,device_id
date,
2020-01-31,80297
2020-02-29,38078
2020-03-31,36222


## 3. Когорты
Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней.

Для какой когорты конверсия была наибольшей?

*считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.*

In [38]:
# создаем переменные cohort(когорту на основе даты регистрации)
cohort_df = df.query('event in ("app_install", "purchase")')[['device_id', 
                                                             'date',
                                                             'event']]

cohort_df = cohort_df.merge(cohort_df.query('event == "app_install"') \
                            .sort_values('date') \
                            .drop_duplicates('device_id')[['device_id', 
                                                           'date']] \
                            .rename(columns = {'date' : 'cohort'}), 
                            how = 'inner', 
                            on  = 'device_id')
cohort_df

,device_id,date,event,cohort
0,6081155,2020-01-01,purchase,2020-01-01
1,6081155,2020-01-01,app_install,2020-01-01
2,77448,2020-01-01,purchase,2020-01-01
3,77448,2020-01-06,purchase,2020-01-01
4,77448,2020-01-13,purchase,2020-01-01
...,...,...,...,...
269817,2261815,2020-03-31,app_install,2020-03-31
269818,19717968,2020-03-31,app_install,2020-03-31
269819,32966665,2020-03-31,app_install,2020-03-31
269820,6335964,2020-03-31,app_install,2020-03-31


In [39]:
cohort_df['period_number'] = (cohort_df.query('event == "purchase"').date - cohort_df.cohort)
cohort_df

,device_id,date,event,cohort,period_number
0,6081155,2020-01-01,purchase,2020-01-01,0 days
1,6081155,2020-01-01,app_install,2020-01-01,NaT
2,77448,2020-01-01,purchase,2020-01-01,0 days
3,77448,2020-01-06,purchase,2020-01-01,5 days
4,77448,2020-01-13,purchase,2020-01-01,12 days
...,...,...,...,...,...
269817,2261815,2020-03-31,app_install,2020-03-31,NaT
269818,19717968,2020-03-31,app_install,2020-03-31,NaT
269819,32966665,2020-03-31,app_install,2020-03-31,NaT
269820,6335964,2020-03-31,app_install,2020-03-31,NaT


In [114]:
size = cohort_df.query('event == "app_install"') \
                .groupby(['cohort']) \
                .agg({'device_id' : 'nunique'}) \
                .reset_index(drop = False)
size.rename(columns = {'device_id' : 'installs'}, inplace = True)

In [108]:
cv = cohort_df.query(' event == "purchase" & period_number <= "7 days" ') \
              .groupby(['cohort'],
                       as_index = False) \
              .agg({'device_id' : 'nunique'})
cv.rename(columns = {'device_id' : 'purchases'}, inplace = True)

In [115]:
cv['convers'] = cv.purchases/size.installs

In [63]:
import matplotlib.pyplot as plt
import plotly.express as px

In [116]:
px.histogram(data_frame=cv, x='cohort', y='convers', nbins = 92)

In [80]:
cv.query(' cohort == "2020-01-09" ')

,cohort,n_customers
8,2020-01-09,712


In [83]:
cohort_df.query('device_id == 77448')

,device_id,date,event,cohort,period_number
2,77448,2020-01-01,purchase,2020-01-01,0 days
3,77448,2020-01-06,purchase,2020-01-01,5 days
4,77448,2020-01-13,purchase,2020-01-01,12 days
5,77448,2020-01-16,purchase,2020-01-01,15 days
6,77448,2020-01-23,purchase,2020-01-01,22 days
7,77448,2020-02-01,purchase,2020-01-01,31 days
8,77448,2020-02-06,purchase,2020-01-01,36 days
9,77448,2020-02-10,purchase,2020-01-01,40 days
10,77448,2020-02-17,purchase,2020-01-01,47 days
11,77448,2020-02-20,purchase,2020-01-01,50 days


In [84]:
cohort_df.query(' event == "purchase" & period_number < "7 days" ')

,device_id,date,event,cohort,period_number
0,6081155,2020-01-01,purchase,2020-01-01,0 days
2,77448,2020-01-01,purchase,2020-01-01,0 days
3,77448,2020-01-06,purchase,2020-01-01,5 days
15,7458607,2020-01-01,purchase,2020-01-01,0 days
17,3520947,2020-01-01,purchase,2020-01-01,0 days
...,...,...,...,...,...
172671,30148472,2020-03-31,purchase,2020-03-31,0 days
172675,8700073,2020-03-31,purchase,2020-03-31,0 days
172677,13278240,2020-03-31,purchase,2020-03-30,1 days
172679,11864116,2020-03-31,purchase,2020-03-25,6 days


## 4. С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [81]:
df.groupby('utm_source', as_index = False).agg({'device_id' : 'nunique'})

,utm_source,device_id
0,-,52273
1,facebook_ads,25959
2,google_ads,38096
3,instagram_ads,31048
4,referal,15926
5,vk_ads,34086
6,yandex-direct,40712


## 5. Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

Подсказка: 

- Для того, чтобы выделить группу тех, кому нужно и не нужно регистрироваться, добавьте колонку с датой регистрации. Если дата регистрации < даты совершения события, то пользователь уже зарегистрирован. 

In [366]:
df.query('event == "register" ')\
  .groupby(['device_id']) \
  .agg({'date' : 'nunique'}) \
  .reset_index() \
  .query("date  > 1")
#убедились, что не бывает двйоных регистраций

,device_id,date


In [447]:
vor_df = df[['event', 
             'device_id', 
             'date']].merge(df.query('event == "register" ')[['date', 
                                                              'device_id']]
                            .rename(columns = {'date' : 'reg_date'}),
                            how = 'left',
                            on  = 'device_id')
vor_df

,event,device_id,date,reg_date
0,app_start,669460,2020-01-01,2020-01-07
1,app_start,833621,2020-01-01,2020-01-01
2,app_start,1579237,2020-01-01,2020-01-01
3,app_start,1737182,2020-01-01,2020-01-01
4,app_start,4029024,2020-01-01,NaN
...,...,...,...,...
2747963,register,2984778,2020-03-31,2020-03-31
2747964,register,27301864,2020-03-31,2020-03-31
2747965,register,1294285,2020-03-31,2020-03-31
2747966,register,3010574,2020-03-31,2020-03-31


In [448]:
reg_df     = vor_df.query('reg_date <= date ')
non_reg_df = vor_df.query('reg_date > date or reg_date != reg_date')

In [449]:
vor_reg_df = reg_df.groupby(['event'], 
                            as_index = False) \
                   .agg({'device_id' : 'nunique'})
vor_reg_df

,event,device_id
0,app_install,42448
1,app_start,78289
2,choose_item,78310
3,purchase,70875
4,register,78310
5,search,78310
6,tap_basket,78310


In [450]:
vor_non_reg_df = non_reg_df.groupby(['event'], 
                                    as_index = False) \
                           .agg({'device_id' : 'nunique'})
vor_non_reg_df

,event,device_id
0,app_install,112149
1,app_start,138482
2,choose_item,98084
3,search,131260
4,tap_basket,61949


In [451]:
vor_reg_df = vor_reg_df.query("event in ('search', 'choose_item', 'tap_basket',  'purchase')")
vor_reg_df = vor_reg_df.loc[[5, 2, 6, 3]]
vor_reg_df

,event,device_id
5,search,78310
2,choose_item,78310
6,tap_basket,78310
3,purchase,70875


In [452]:
vor_non_reg_df = vor_non_reg_df.query("event in ('search', 'choose_item', 'tap_basket',  'purchase')")
vor_non_reg_df = vor_non_reg_df.loc[[3, 2, 4]]
vor_non_reg_df
#нет покупок у незарегестрированных пользователей

,event,device_id
3,search,131260
2,choose_item,98084
4,tap_basket,61949


In [453]:
vor_reg_df['cr_by_step'] = vor_reg_df.device_id.div(vor_reg_df.device_id.shift(1))
vor_reg_df

,event,device_id,cr_by_step
5,search,78310,NaN
2,choose_item,78310,1.000000
6,tap_basket,78310,1.000000
3,purchase,70875,0.905057


## 6. Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?
 
- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

In [367]:
df.query('event == "app_install" ')\
  .groupby(['device_id']) \
  .agg({'date' : 'nunique'}) \
  .reset_index() \
  .query("date  > 1")
#убедились, что не бывает двйоных установок

,device_id,date


In [443]:
conv_df = df[['event', 
             'device_id', 
             'date',
             'utm_source']].merge(df.query('event == "app_install"')[['date', 
                                                              'device_id']]
                            .rename(columns = {'date' : 'app_install_date'}),
                            how = 'inner',
                            on  = 'device_id') \
            .query('event in ("app_install", "purchase") & date >= app_install_date  & utm_source != "-"')
conv_df

,event,device_id,date,utm_source,app_install_date
50,purchase,833621,2020-01-01,vk_ads,2020-01-01
62,purchase,833621,2020-01-11,referal,2020-01-01
77,purchase,833621,2020-01-16,referal,2020-01-01
110,purchase,833621,2020-02-02,referal,2020-01-01
144,purchase,833621,2020-02-25,referal,2020-01-01
...,...,...,...,...,...
2263005,app_install,908531,2020-03-31,yandex-direct,2020-03-31
2263010,app_install,3925522,2020-03-31,instagram_ads,2020-03-31
2263012,app_install,6369878,2020-03-31,google_ads,2020-03-31
2263017,app_install,18983670,2020-03-31,instagram_ads,2020-03-31


In [444]:
cr_utm = conv_df.query('event == "purchase"') \
                .groupby(['utm_source'], as_index = False) \
                .agg({'device_id' : 'nunique'})
cr_utm.rename(columns = {'device_id' : 'count_first_purchase'}, 
              inplace = True)
cr_utm

,utm_source,count_first_purchase
0,facebook_ads,7227
1,google_ads,9865
2,instagram_ads,8720
3,referal,4919
4,vk_ads,10010
5,yandex-direct,10495


In [445]:
inst_utm = conv_df.query('event == "app_install"') \
                  .groupby(['utm_source'], as_index = False) \
                  .agg({'device_id' : 'nunique'})
inst_utm.rename(columns = {'device_id' : 'count_installs'},
                inplace = True)
inst_utm

,utm_source,count_installs
0,facebook_ads,13916
1,google_ads,26286
2,instagram_ads,20096
3,referal,9282
4,vk_ads,23189
5,yandex-direct,29368


In [446]:
cr_utm['cr'] = cr_utm.count_first_purchase/inst_utm.count_installs
cr_utm[cr_utm.cr == cr_utm.cr.min()]

,utm_source,count_first_purchase,cr
5,yandex-direct,10495,0.357362


## 7. Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? 
- учитываются только первые покупки пользователей

In [514]:
df_chek = df.query('utm_source != "-" & event == "purchase" ')[['date', 
                                                                'purchase_sum', 
                                                                'device_id', 
                                                                'utm_source']]

In [515]:
#проверим были ли покупки в один день
df_chek.groupby(['device_id', 'date']) \
       .agg({'purchase_sum' : 'count'}) \
       .reset_index() \
       .query('purchase_sum > 1')

,device_id,date,purchase_sum


In [540]:
# groupby сортирует по возростанию даты
df_first_chek = df_chek.sort_values(['device_id', 
                                     'date']) \
                       .drop_duplicates('device_id')[['purchase_sum', 
                                                      'utm_source']] \
                       .groupby('utm_source') \
                       .median() \
                       .reset_index()
df_first_chek

,utm_source,purchase_sum
0,facebook_ads,389.25
1,google_ads,389.50
2,instagram_ads,393.50
3,referal,395.50
4,vk_ads,392.50
5,yandex-direct,393.00


In [541]:
df_first_chek.loc[[df_first_chek.purchase_sum.idxmax()]]

,utm_source,purchase_sum
3,referal,395.5
